# Numba demos snippets

Numba is a compiler for Python array and numerical functions.

## How does Numba work?
Numba reads the Python bytecode for a decorated function and combines this with information about the types of the input arguments to the function. It analyzes and optimizes your code, and finally uses the LLVM compiler library to generate a machine code version of your function, tailored to your CPU capabilities. This compiled version is then used every time your function is called.


In [ ]:
import time

import numba
import numpy as np
import pandas as pd
from numba import config, jit, njit, threading_layer

In [ ]:
x = np.arange(100).reshape(10, 10)


@jit(nopython=True)
def go_fast(a):  # Function is compiled and runs in machine code
    trace = 0.0
    for i in range(a.shape[0]):
        trace += np.tanh(a[i, i])
    return a + trace


# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
go_fast(x)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
go_fast(x)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

In [ ]:
# Even better time recording:
# measure multiple iterations of execution and, as a result,
# can be made to accommodate for the compilation time in the first execution.
%timeit go_fast(x)

In [ ]:
# simple example with pandas DataFrames


def f(x):
    return x * (x - 1)


def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx


@jit
def f_plain(x):
    return x * (x - 1)


@jit
def integrate_f_numba(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f_plain(a + i * dx)
    return s * dx


@jit(nopython=True)
def apply_integrate_f_numba(col_a, col_b, col_N):
    n = len(col_N)
    result = np.empty(n, dtype=np.float64)
    assert len(col_a) == len(col_b) == n
    for i in range(n):
        result[i] = integrate_f_numba(col_a[i], col_b[i], col_N[i])
    return result


def compute_numba(df):
    result = apply_integrate_f_numba(df["a"].to_numpy(), df["b"].to_numpy(), df["N"].to_numpy())
    return pd.Series(result, index=df.index, name="result")

In [ ]:
rand_df = pd.DataFrame(
    {"a": np.random.randn(1000), "b": np.random.randn(1000), "N": np.random.randint(100, 1000, (1000)), "x": "x"}
)

In [ ]:
%timeit rand_df.apply(lambda x: integrate_f(x['a'], x['b'], x['N']), axis=1)

In [ ]:
%timeit compute_numba(rand_df)

## Parallel logistic regression


In [ ]:
@numba.jit(nopython=True, parallel=True)
def logistic_regression(Y, X, w, iterations):
    for i in range(iterations):
        w -= np.dot(((1.0 / (1.0 + np.exp(-Y * np.dot(X, w))) - 1.0) * Y), X)
    return w

## Parallel execution

In [ ]:
# set the threading layer before any parallel target compilation
config.THREADING_LAYER = "threadsafe"


@njit(parallel=True)
def foo(a, b):
    return a + b


x = np.arange(10.0)
y = x.copy()

# this will force the compilation of the function, select a threading layer
# and then execute in parallel
foo(x, y)

# demonstrate the threading layer chosen
print("Threading layer chosen: %s" % threading_layer())